In [1]:
import spacy

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

2022-06-05 19:27:56.641140: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-05 19:27:56.641178: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
corpus = fetch_20newsgroups(categories=['sci.space'], 
                            remove=('headers', 'footers', 'quotes'))

In [3]:
print(type(corpus))

<class 'sklearn.utils.Bunch'>


In [4]:
len(corpus.data)

593

In [5]:
corpus.data[:2]

["\nAny lunar satellite needs fuel to do regular orbit corrections, and when\nits fuel runs out it will crash within months.  The orbits of the Apollo\nmotherships changed noticeably during lunar missions lasting only a few\ndays.  It is *possible* that there are stable orbits here and there --\nthe Moon's gravitational field is poorly mapped -- but we know of none.\n\nPerturbations from Sun and Earth are relatively minor issues at low\naltitudes.  The big problem is that the Moon's own gravitational field\nis quite lumpy due to the irregular distribution of mass within the Moon.",
 '\nGlad to see Griffin is spending his time on engineering rather than on\nritual purification of the language.  Pity he got stuck with the turkey\nrather than one of the sensible options.']

In [8]:
# Like before, if we want to use spaCy's tokenizer, we need 
# to create a callback. Remember to upgrade spaCy if you need 
# to (refer to beginnning of file for commentary and instructions).
nlp = spacy.load('en_core_web_sm')

# We don't need named-entity recognition nor dependency parsing for
# this so these components are disabled. This will speed up the 
# pipeline. We do need part-of-speech tagging however.
unwanted_pipes = ["ner", "parser"]

# For this exercise, we'll remove punctuation and spaces (which
# includes newlines), filter for tokens consisting of alphabetic
# characters, and return the lemma (which require POS tagging).
def spacy_tokenizer(doc):
  with nlp.disable_pipes(*unwanted_pipes):
    return [t.lemma_ for t in nlp(doc) if \
            not t.is_punct and \
            not t.is_space and \
            t.is_alpha]

In [9]:
%%time
# Use the default settings of TfidfVectorizer.
vectorizer = TfidfVectorizer(tokenizer=spacy_tokenizer)
features = vectorizer.fit_transform(corpus.data)

CPU times: user 14.4 s, sys: 222 ms, total: 14.6 s
Wall time: 14.6 s


In [10]:
# The number of unique tokens.
print(len(vectorizer.get_feature_names_out()))

9469


In [11]:
# The dimensions of our feature matrix. X rows (documents) by Y columns (tokens).
print(features.shape)

(593, 9469)


In [12]:
# What the encoding of the first document looks like in sparse format.
print(features[0])

  (0, 5076)	0.10452465319204224
  (0, 2355)	0.12746673572641337
  (0, 4351)	0.15331277268825122
  (0, 2464)	0.10862134983649849
  (0, 4927)	0.17102243214182047
  (0, 6721)	0.09939758959196421
  (0, 5997)	0.10183273017124134
  (0, 6531)	0.08455248650428543
  (0, 903)	0.08929749232550656
  (0, 321)	0.11094564582599897
  (0, 4907)	0.0824741348739743
  (0, 637)	0.05104326044583604
  (0, 4378)	0.10269890253976902
  (0, 5285)	0.13259379932649137
  (0, 6928)	0.12524362655380208
  (0, 2501)	0.07376358403679217
  (0, 8137)	0.09512941822420008
  (0, 3299)	0.051873252060803496
  (0, 6196)	0.13901479196044814
  (0, 5662)	0.11219221685212125
  (0, 4604)	0.06321553828701997
  (0, 9188)	0.060883075560077576
  (0, 1148)	0.048917557559216875
  (0, 5035)	0.12319856435864923
  (0, 6371)	0.15331277268825122
  :	:
  (0, 1351)	0.09036221462758656
  (0, 5413)	0.17102243214182047
  (0, 456)	0.10452465319204224
  (0, 5802)	0.09913077191467295
  (0, 8399)	0.20402427950185778
  (0, 5387)	0.10099496207230896
  (0

In [13]:
# Transform the query into a TF-IDF vector.
query = ["lunar orbit"]
query_tfidf = vectorizer.transform(query)

In [14]:
# Calculate the cosine similarities between the query and each document.
# We're calling flatten() here becaue cosine_similarity returns a list
# of lists and we just want a single list.
cosine_similarities = cosine_similarity(features, query_tfidf).flatten()

In [15]:
import numpy as np

# numpy's argsort() method returns a list of *indices* that
# would sort an array:
# https://numpy.org/doc/stable/reference/generated/numpy.argsort.html
#
# The sort is ascending, but we want the largest k cosine_similarites
# at the bottom of the sort. So we negate k, and get the last k
# entries of the indices list in reverse order. There are faster
# ways to do this using things like argpartition but this is
# more succinct.
def top_k(arr, k):
  kth_largest = (k + 1) * -1
  return np.argsort(arr)[:kth_largest:-1]

In [16]:
# So for our query above, these are the top five documents.
top_related_indices = top_k(cosine_similarities, 5)
print(top_related_indices)

[249 108   0 312 509]


In [17]:
# Let's take a look at their respective cosine similarities.
print(cosine_similarities[top_related_indices])

[0.4784239  0.42898437 0.27362524 0.19484222 0.19133134]


In [18]:
# Top match.
print(corpus.data[top_related_indices[0]])


Actually, Hiten wasn't originally intended to go into lunar orbit at all,
so it indeed didn't have much fuel on hand.  The lunar-orbit mission was
an afterthought, after Hagoromo (a tiny subsatellite deployed by Hiten
during a lunar flyby) had a transmitter failure and its proper insertion
into lunar orbit couldn't be positively confirmed.

It should be noted that the technique does have disadvantages.  It takes
a long time, and you end up with a relatively inconvenient lunar orbit.
If you want something useful like a low circular polar orbit, you do have
to plan to expend a certain amount of fuel, although it is reduced from
what you'd need for the brute-force approach.


In [19]:
# Second-best match.
print(corpus.data[top_related_indices[1]])


Their Hiten engineering-test mission spent a while in a highly eccentric
Earth orbit doing lunar flybys, and then was inserted into lunar orbit
using some very tricky gravity-assist-like maneuvering.  This meant that
it would crash on the Moon eventually, since there is no such thing as
a stable lunar orbit (as far as anyone knows), and I believe I recall
hearing recently that it was about to happen.


In [20]:
# Try a different query
query = ["satellite"]
query_tfidf = vectorizer.transform(query)

cosine_similarities = cosine_similarity(features, query_tfidf).flatten()
top_related_indices = top_k(cosine_similarities, 5)

print(top_related_indices)
print(cosine_similarities[top_related_indices])

[378 138 248  61 236]
[0.38931334 0.34090226 0.29842859 0.25673657 0.24577701]


In [25]:
for i in top_related_indices:
    print(corpus.data[i])
    print('********************')



As an Amateur Radio operator (VHF 2metres) I like to keep up with what is 
going up (and for that matter what is coming down too).
 
In about 30 days I have learned ALOT about satellites current, future and 
past all the way back to Vanguard series and up to Astro D observatory 
(space).  I borrowed a book from the library called Weater Satellites (I 
think, it has a photo of the earth with a TIROS type satellite on it.)
 
I would like to build a model or have a large color poster of one of the 
TIROS satellites I think there are places in the USA that sell them.
ITOS is my favorite looking satellite, followed by AmSat-OSCAR 13 
(AO-13).
 
TTYL
73
Jim
********************
I've just read Richard Langley's latest "Navstar GPS Constellation Status".

It states that the latest satellite was placed in Orbit Plane Position C-3.
There is already one satellite in that position. I know that it's almost
ten years since that satellite was launched but it's still in operation so
why not use it u